<h1><b>Statistique en Bioinformatique : </b> TME8 </h1><br>

L’objectif de ce TME est: 
<br>
<ul>
<li> Evaluer la performance des HMMs sur les homologues lointains </li> 
</ul>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p>**Soumission**</p>
<ul>
<li>Renomer le fichier TME9.ipynb pour NomEtudiant1_NomEtudiant2_TME9.ipynb </li>
<li>Envoyer par email à edoardo.sarti@upmc.fr, l’objet de l'email sera [SBAS-2019] TME9</li>
</ul>
</div>

Nom etudiant 1 :
Yann Zhong

Nom etudiant 2 :
Alexis Trang

<h2> Datasets </h2><br>
To evaluate the performance of HMMer over remote homologous proteins, we will work with datasets scop-95 and scop-30 having at most 95 and 30% of identity.  We will use a leave-one-out strategy as follows. Given a scop family $F$  in one of the scop datasets, we consider the set of $n$ sequences associated to $F$ in scop, to create $n$ test-sets for $F$. Each test-set takes $n−1$ sequences for training and leaves one sequence out for the test. Then, we test whether the sequence that was left out could be annotated by a HMMer profile constructed without using it, and count the correct identification of the domain as a true positive (TP), the identification of an erroneous domain as a false positive (FP) and the identification of no domain as a false negative (FN). For each scop dataset you can find the directory `aln` that contains the alignment for each testsets in STOCKHOLM format, extension .sto

For scop-95 dataset do:

1\. Create pHMMs from the alignments in `aln/*.sto`. Use the program `hmmbuild` from the HMMer-3 package. Save models  in a directory named models. Put your code below, but run it on the terminal.

```bash
#Création du dossier models en partant du dossier TME8
cd ~/Documents/SBAS_TME/TME8
mkdir ./models
( mkdir models ) #Works too

#Tout en étant dans le même dossier TME8, on va dans le dossier aln où se trouve les fichiers .sto
cd scop/scop/scop95/aln

#Boucle pour faire le hmmbuild et créer les fichiers hmm
for file in `ls`; do hmmbuild $file.hmm $file; done;

#Les fichiers hmm ont été crées dans le dossier aln et on les déplace dans le dossier models crée au dessus
mv *.hmm ~/Documents/SBAS_TME/TME8/models
```

2\. Search all sequences in the file `scopTestSeq.fasta` against all models created previously. Use `hmmsearch` and the option `--domtblout` to save parseable table of per-domain hits to file, use option `-E 1`. Save the outputs in a directory named `searchResults/`. Put your code below, but run it on the terminal.

```bash
#Création du dossier searchResults en partant du dossier TME8
cd ~/Documents/SBAS_TME/TME8
mkdir ./searchResults

#Copie du fichier scopTestSeq.fasta dans le dossier models 
cp scop/scop/scopTestSeq.fasta ~/Documents/SBAS_TME/TME8/models

#On place le terminal dans le dossier models avec les fichiers hmm et le fichier fasta
cd ~/Documents/SBAS_TME/TME8/models

#Boucle pour faire le hmmsearch
for file in `ls -I scopTestSeq.fasta`; do hmmsearch -E 1 --domtblout $file.out $file scopTestSeq.fasta; done

#On déplace les fichiers out dans le dossier searchResults
mv *.out ~/Documents/SBAS_TME/TME8/searchResults

#On supprime le fichier fichier fasta que l'on a ajouté dans le dossier models
rm scopTestSeq.fasta 
```

3\. Concat all search results, rank the sequences and remove all hits having same family and different proteinID. Consider just the best hit per sequence.

```bash
# the following bash commands illustrate a way to concatenate and format the output of hmmsearch
# hmmsearch results are assumed to be in the current directory and in files with extension .out
cat *.out > allRes.txt
sed 's/^#/d' allRes.txt > allRes.txt.ftt
cat allRes.txt.ftt | awk '{print $1" "$4" "$12}' > allRes.txt.ftt.2
sed -i -e 's/\.aln//g' allRes.txt.ftt.2
mv allRes.txt.ftt.2 allRes.txt.ftt
```

```bash
# the following bash commands illustrate a way to concatenate and format the output of hmmsearch
# hmmsearch results are assumed to be in the current directory and in files with extension .out
cat *.out > allRes.txt
sed '/^#/d' allRes.txt > allRes.txt.ftt
cat allRes.txt.ftt | awk '{print $1" "$4" "$12}' > allRes.txt.ftt.2
sed -i -e 's/\.aln//g' allRes.txt.ftt.2
mv allRes.txt.ftt.2 allRes.txt.ftt
```

4\. Compute the performance. If a test sequence is better scored by the model associated to its family it is a true positive (TP). If it is scored by a different model it is a false positive (FP). If it is not scored by any model it is a false negative (FN). With this values compute $\text{Precision}=\frac{TP}{TP+FP}$, $\text{Recall} = \frac{TP}{TP+FN}$, and $\text{F-score} = \frac{ 2 \cdot \text{Precision} \cdot \text{Recall} }{ \text{Precision} + \text{Recall} }$.

5\.Roc curve https://en.wikipedia.org/wiki/Receiver_operating_characteristic. We can plot curves to observe the performance of a tool under different conditions. For that, we range the score (E-value) and we compute true positive rate (Y axis) and false positive rate (X-axis) measures for each value. Write a python script to construct the Roc curve based on the results obtained before.

6\. Write a python script to plot the Precision/recall curve. We can use the same strategy of Roc curve to compute the precision  (Y axis)  and recall  (X axis)  curves.

7\. Repeat steps 1 to 6 also for the scop-30 dataset and compare the curves obtained for the two SCOP datasets, what is your conclusion? Include the curves in your answer.